# Result Replication

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

Pokusati ću replicirati rezultate iz članka.

In [34]:
df = pd.read_csv("march-machine-learning-mania-2015/regular_season_detailed_results.csv", sep = ",")

## Izračun RPI
Moramo izračunati RPI (Rating percentage index) koji se koristio od 1981 do 2020. Zamjenio ga je BPI (Basketball Power Index) koji je osmislio ESPN. Formula za BPI nije poznata te je poslovna tajna ESPN-a.

In [35]:
def calculate_rpi(df):
  # Calculate winning percentage for each team
  wins = df['wteam'].value_counts()
  losses = df['lteam'].value_counts()
  games_played = wins.add(losses, fill_value=0)
  wp = wins / games_played

  # Calculate opponents winning percentage
  opponents_wps = pd.concat([df.groupby('wteam')['lteam'].apply(lambda x: wp[x].mean()),
                 df.groupby('lteam')['wteam'].apply(lambda x: wp[x].mean())])
  owp = opponents_wps.groupby(opponents_wps.index).mean()

  # Calculate opponents opponents winning percentage
  opponents_owps = pd.concat([df.groupby('wteam')['lteam'].apply(lambda x: owp[x].mean()),
                df.groupby('lteam')['wteam'].apply(lambda x: owp[x].mean())])
  oowp = opponents_owps.groupby(opponents_owps.index).mean()

  # Calculate RPI
  rpi = (wp * 0.25) + (owp * 0.50) + (oowp * 0.25)
  return rpi

## Algoritmi iz članka
U članku su navedena dva algoritma koja su se koristila za procjenu.

In [ ]:
# Function to calculate probability of winning
def calProbWin(D_v, w_v, l_v, f_v):
    x = abs(w_v - l_v)
    y = f_v.predict(np.array([x]).reshape(-1, 1))
    if (w_v - l_v) > 0:
        return y
    else:
        return 1 - y


def predictWin(D_prime, V_prime):
    # D_prime is your DataFrame
    # V_prime is the list of predictor variables

    # Define the response variable and predictor variables
    X = D_prime[V_prime]
    y = D_prime['rpi_wteam']  # assuming 'win' is your response variable

    # Add a constant to the predictor variables
    X = sm.add_constant(X)

    # Fit the model
    model = sm.OLS(y, np.asarray(X)).fit()

    # Make predictions
    predictions = model.predict(X)

    # Create a DataFrame with the IDs and predictions
    results = pd.DataFrame({
        'id': D_prime['season'].astype(str) + '_' + D_prime['wteam'].astype(str) + '_' + D_prime['lteam'].astype(str),
        'pred': predictions
    })

    return results


## Procjena

In [36]:
# Convert columns to numeric data types
# df['wteam'] = pd.to_numeric(df['wteam'], errors='coerce')
# df['lteam'] = pd.to_numeric(df['lteam'], errors='coerce')

# Calculate RPI for each team
rpi = calculate_rpi(df)

# Add the calculated RPIs to the dataset
df['rpi_wteam'] = df['wteam'].map(rpi)
df['rpi_lteam'] = df['lteam'].map(rpi)

# Convert RPI columns to numeric data types
# df['rpi_wteam'] = pd.to_numeric(df['rpi_wteam'], errors='coerce')
# df['rpi_lteam'] = pd.to_numeric(df['rpi_lteam'], errors='coerce')

# Call predictWin function
P = predictWin(df, df.keys())

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).